In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('t20i_info.csv')

In [3]:
df.head()

,Unnamed: 0,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,0,2,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,1,2,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground
2,2,2,Australia,Sri Lanka,0.3,1,0,NaN,Melbourne Cricket Ground
3,3,2,Australia,Sri Lanka,0.4,2,0,NaN,Melbourne Cricket Ground
4,4,2,Australia,Sri Lanka,0.5,0,0,NaN,Melbourne Cricket Ground


In [4]:
df.shape

(63888, 9)

In [5]:
# Count the unique venue values where city is null to understand the pattern of missing city values
df[df['city'].isnull()]['venue'].value_counts()

venue
Dubai International Cricket Stadium        2969
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sharjah Cricket Stadium                     249
Sylhet International Cricket Stadium        128
Carrara Oval                                 64
Name: count, dtype: int64

In [6]:
cities = np.where(
    df['city'].isnull(),  # condition: where 'city' is null
    df['venue'].str.split().apply(lambda x: x[0]),  # value to use when 'city' is null: first word of 'enue'
    df['city']  # value to use when 'city' is not null: original 'city' value
)

In [7]:
df['city']=cities

In [8]:
df.isnull().sum()

Unnamed: 0          0
match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [9]:
#Drop unwanted columns
df.drop(columns=['Unnamed: 0', 'venue'], inplace=True)

In [10]:
#DataFrame containing data
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne
...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo


In [11]:
eligible_cities = df['city'].value_counts()[df['city'].value_counts()>600].index.tolist()

In [12]:
df=df[df['city'].isin(eligible_cities)]

In [13]:
df['current_score'] = df.groupby('match_id')['runs'].cumsum()

In [14]:
#see the resulting dataframe df after running the code!
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3
...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo,125
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo,125
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo,127


In [15]:
#Now our next target is to create a ‘balls_left’ column for which firstly we would be creating to new columns: ‘overs’ and ‘balls’ which tells us how many overs have been completed and how many balls of the current over has been bowled respectively. The code is very simple for that.

In [16]:
df['over'] = df['ball'].apply(lambda x: str(x).split(".")[0])
df['ball_no'] = df['ball'].apply(lambda x: str(x).split(".")[1])
df['balls_left'] = 6 - df['ball_no'].astype(int)

In [17]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_left
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,5
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,4
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,2
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo,125,19,3,3
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo,125,19,4,2
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,19,5,1
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo,127,19,6,0


###Now by using a simple formula we can create a ‘balls_bowled’ column that is how many balls have been bowled. Formula would be
  ,.. balls_bowled = (overs * 6) + balls

In [18]:
df['balls_bowled']=(df['over'].astype('int')*6) + df['ball_no'].astype('int')

In [19]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_left,balls_bowled
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,5,1
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,4,2
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,3
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,2,4
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo,125,19,3,3,117
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo,125,19,4,2,118
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,19,5,1,119
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo,127,19,6,0,120


This code calculates the number of balls left in a cricket innings by subtracting the number of balls
already bowled from the total number of balls (120) and ensures that the result is not negative
Calculate balls left in cricket innings

In [20]:
df['balls_left'] = 120 - df['balls_bowled']
df['balls_left'] = df['balls_left'].apply(lambda x: 0 if x < 0 else x)

In [21]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_left,balls_bowled
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,119,1
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,118,2
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,117,3
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,116,4
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,115,5
...,...,...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo,125,19,3,3,117
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo,125,19,4,2,118
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,19,5,1,119
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo,127,19,6,0,120


Now if we look at the ‘player_dismissed’ column it has either value 0 or
name of the player got out at that particular ball. First we will replace 
all the names with 1 and then apply the cumsum() function on it so we can get the
total wickets gone and we will subtract it from 10 to get the ‘wickets_left’ column.

In [22]:
df['player_dismissed'] = df['player_dismissed'].apply(lambda x: 0 if x == '0' else 1)
df['player_dismissed'] = df['player_dismissed'].astype(int)

grouped = df.groupby('match_id')
df['player_dismissed'] = grouped['player_dismissed'].cumsum()
df['wickets_left'] = 10 - df['player_dismissed']

In [23]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_left,balls_bowled,wickets_left
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,119,1,10
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,118,2,10
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,117,3,10
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,116,4,10
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,115,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,8,Colombo,125,19,3,3,117,2
63884,964,Sri Lanka,Australia,19.4,0,8,Colombo,125,19,4,2,118,2
63885,964,Sri Lanka,Australia,19.5,0,9,Colombo,125,19,5,1,119,1
63886,964,Sri Lanka,Australia,19.6,2,9,Colombo,127,19,6,0,120,1


In [24]:
df['crr'] = ((df['current_score'] * 6) / df['balls_bowled']).round(2)

In [25]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_left,balls_bowled,wickets_left,crr
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,119,1,10,0.00
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,118,2,10,0.00
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,117,3,10,2.00
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,116,4,10,4.50
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,115,5,10,3.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,8,Colombo,125,19,3,3,117,2,6.41
63884,964,Sri Lanka,Australia,19.4,0,8,Colombo,125,19,4,2,118,2,6.36
63885,964,Sri Lanka,Australia,19.5,0,9,Colombo,125,19,5,1,119,1,6.30
63886,964,Sri Lanka,Australia,19.6,2,9,Colombo,127,19,6,0,120,1,6.35


 Now we need a column that has total runs scored in last five overs.
Obviously we will have null values in this column for first 5 overs

In [26]:
last_five = df.groupby('match_id')['runs'].rolling(window=30).sum().tolist()

In [27]:
df['last_five']=last_five

In [28]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_left,balls_bowled,wickets_left,crr,last_five
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,119,1,10,0.00,NaN
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,118,2,10,0.00,NaN
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,117,3,10,2.00,NaN
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,116,4,10,4.50,NaN
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,115,5,10,3.60,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,8,Colombo,125,19,3,3,117,2,6.41,32.0
63884,964,Sri Lanka,Australia,19.4,0,8,Colombo,125,19,4,2,118,2,6.36,32.0
63885,964,Sri Lanka,Australia,19.5,0,9,Colombo,125,19,5,1,119,1,6.30,32.0
63886,964,Sri Lanka,Australia,19.6,2,9,Colombo,127,19,6,0,120,1,6.35,33.0


In [29]:
df.head(40)

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_left,balls_bowled,wickets_left,crr,last_five
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,119,1,10,0.00,NaN
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,118,2,10,0.00,NaN
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,117,3,10,2.00,NaN
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,116,4,10,4.50,NaN
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,115,5,10,3.60,NaN
5,2,Australia,Sri Lanka,0.6,3,0,Melbourne,6,0,6,114,6,10,6.00,NaN
6,2,Australia,Sri Lanka,1.1,0,0,Melbourne,6,1,1,113,7,10,5.14,NaN
7,2,Australia,Sri Lanka,1.2,1,0,Melbourne,7,1,2,112,8,10,5.25,NaN
8,2,Australia,Sri Lanka,1.3,0,0,Melbourne,7,1,3,111,9,10,4.67,NaN
9,2,Australia,Sri Lanka,1.4,0,0,Melbourne,7,1,4,110,10,10,4.20,NaN


Now we have to create a last column which would be our target column. Total runs scored in that innings

In [30]:
#final_df=df.groupby('match_id').sum()['runs'].reset_index().merge(df,on='match_id')

In [31]:
#final_df=final_df[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','crr','last_five','runs_x']]

In [32]:
#final_df

In [33]:
# Ensure the 'match_id' column is of a suitable type for grouping
df['match_id'] = df['match_id'].astype(str)

# Create a new column 'total_runs' with the total runs for each match
df['total_runs'] = df.groupby('match_id')['runs'].transform('sum')

# Select the desired columns for the final dataframe
final_df = df[['batting_team', 'bowling_team', 'city', 'current_score', 'balls_left', 'wickets_left', 'crr', 'last_five',  'total_runs']]

In [34]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,total_runs
0,Australia,Sri Lanka,Melbourne,0,119,10,0.00,NaN,168
1,Australia,Sri Lanka,Melbourne,0,118,10,0.00,NaN,168
2,Australia,Sri Lanka,Melbourne,1,117,10,2.00,NaN,168
3,Australia,Sri Lanka,Melbourne,3,116,10,4.50,NaN,168
4,Australia,Sri Lanka,Melbourne,3,115,10,3.60,NaN,168
...,...,...,...,...,...,...,...,...,...
63883,Sri Lanka,Australia,Colombo,125,3,2,6.41,32.0,128
63884,Sri Lanka,Australia,Colombo,125,2,2,6.36,32.0,128
63885,Sri Lanka,Australia,Colombo,125,1,1,6.30,32.0,128
63886,Sri Lanka,Australia,Colombo,127,0,1,6.35,33.0,128


In [35]:
# Check for missing values
print("Missing values before dropping:")
print(final_df.isnull().sum())

# Drop rows with missing values
final_df.dropna(inplace=True)

# Verify that there are no missing values left
print("Missing values after dropping:")
print(final_df.isnull().sum())

# Shuffle the rows
final_df = final_df.sample(frac=1, random_state=42)  # Set a fixed seed for reproducibility

print("Dataframe shape after shuffling:", final_df.shape)
print("Final dataframe:")
print(final_df.head())  # Show the first few rows of the shuffled dataframe

Missing values before dropping:
batting_team         0
bowling_team         0
city                 0
current_score        0
balls_left           0
wickets_left         0
crr                  0
last_five        12024
total_runs           0
dtype: int64
Missing values after dropping:
batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
total_runs       0
dtype: int64
Dataframe shape after shuffling: (38477, 9)
Final dataframe:
      batting_team bowling_team          city  current_score  balls_left  \
22174    Sri Lanka      England   Southampton            140          22   
31691    Australia     Pakistan         Dubai             97          24   
38313     Pakistan  New Zealand  Christchurch             98          46   
63091   Bangladesh    Sri Lanka        Mirpur             40          85   
19096      England    Australia   Southampton            130          15   

       wickets_

In [36]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,total_runs
22174,Sri Lanka,England,Southampton,140,22,3,8.57,41.0,163
31691,Australia,Pakistan,Dubai,97,24,2,6.06,20.0,108
38313,Pakistan,New Zealand,Christchurch,98,46,8,7.95,21.0,183
63091,Bangladesh,Sri Lanka,Mirpur,40,85,7,6.86,39.0,147
19096,England,Australia,Southampton,130,15,5,7.43,45.0,145
...,...,...,...,...,...,...,...,...,...
12066,New Zealand,Sri Lanka,Auckland,29,91,6,6.00,29.0,179
22738,New Zealand,Sri Lanka,Wellington,80,75,9,10.67,50.0,162
63482,Sri Lanka,Pakistan,Mirpur,65,62,10,6.72,43.0,150
1787,West Indies,India,Lauderhill,84,83,10,13.62,67.0,245


 we end out feature extraction part of the project

In [37]:
final_df.head(20)

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,total_runs
22174,Sri Lanka,England,Southampton,140,22,3,8.57,41.0,163
31691,Australia,Pakistan,Dubai,97,24,2,6.06,20.0,108
38313,Pakistan,New Zealand,Christchurch,98,46,8,7.95,21.0,183
63091,Bangladesh,Sri Lanka,Mirpur,40,85,7,6.86,39.0,147
19096,England,Australia,Southampton,130,15,5,7.43,45.0,145
25103,India,England,Durban,67,75,10,8.93,51.0,218
49069,West Indies,England,Barbados,111,39,8,8.22,43.0,170
4428,West Indies,Afghanistan,St Kitts,78,56,8,7.31,26.0,112
59799,England,West Indies,Mumbai,45,89,9,8.71,44.0,182
25892,India,Pakistan,Johannesburg,109,33,7,7.52,44.0,157


now begin with model building process. For that first we will divide our dataset in training set and
testing set using train_test_split module of sklearn library

In [38]:
#contains the feature columns for the training set
#X_train

In [39]:
from sklearn.model_selection import train_test_split

# Assuming final_df is your preprocessed DataFrame with features and target (total_runs)
X = final_df.drop(columns=['total_runs'])
y = final_df['total_runs']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
X_train

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five
33592,Pakistan,England,Barbados,47,81,9,7.23,31.0
61405,South Africa,West Indies,Nagpur,47,71,6,5.76,27.0
56921,Australia,South Africa,Durban,91,56,7,8.53,29.0
15427,South Africa,India,Chandigarh,129,11,5,7.10,31.0
31838,Bangladesh,West Indies,St Kitts,117,0,1,5.85,31.0
...,...,...,...,...,...,...,...,...
33399,South Africa,Afghanistan,Barbados,99,27,5,6.39,22.0
8234,Sri Lanka,India,Colombo,109,53,7,9.76,54.0
53021,Bangladesh,Australia,Mirpur,21,87,8,3.82,20.0
41074,West Indies,Bangladesh,Mirpur,129,4,3,6.67,33.0


In [41]:
from sklearn.model_selection import train_test_split

# Assuming final_df is your preprocessed DataFrame with features and target (total_runs)
X = final_df.drop(columns=['total_runs'])
y = final_df['total_runs']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print column names of X_train and X_test to check for consistency
print("Columns in X_train:", X_train.columns)
print("Columns in X_test:", X_test.columns)

# Optionally, print the first few rows to inspect the data
print("Head of X_train:")
print(X_train.head())

print("Head of X_test:")
print(X_test.head())


Columns in X_train: Index(['batting_team', 'bowling_team', 'city', 'current_score', 'balls_left',
       'wickets_left', 'crr', 'last_five'],
      dtype='object')
Columns in X_test: Index(['batting_team', 'bowling_team', 'city', 'current_score', 'balls_left',
       'wickets_left', 'crr', 'last_five'],
      dtype='object')
Head of X_train:
       batting_team  bowling_team        city  current_score  balls_left  \
33592      Pakistan       England    Barbados             47          81   
61405  South Africa   West Indies      Nagpur             47          71   
56921     Australia  South Africa      Durban             91          56   
15427  South Africa         India  Chandigarh            129          11   
31838    Bangladesh   West Indies    St Kitts            117           0   

       wickets_left   crr  last_five  
33592             9  7.23       31.0  
61405             6  5.76       27.0  
56921             7  8.53       29.0  
15427             5  7.10       31.0  
3183

Preprocessing: One-Hot Encoding for categorical features, creating a pipeline with the ML model, and scaling the data

Importing XGBoost library and initializing the model from xgboost import XGBClassifier xgb_model = XGBClassifier()

In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error

In [55]:
transformer = ColumnTransformer([
    ('transformer', OneHotEncoder(sparse_output=False, drop='first'),['batting_team','bowling_team', 'city'])
], remainder='passthrough')

In [56]:
pipe = Pipeline(steps=[
    ('step1', transformer),
    ('step2', StandardScaler()),
    ('step3', XGBRegressor(n_estimators=1000, learning_rate=0.2, max_depth=12, random_state=1))
])

In [57]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

In [58]:
r2_score(y_test, y_pred)

0.9876448909582451

In [59]:
mean_absolute_error(y_test, y_pred)

1.6522092749819686

In [60]:
import pickle

# Assuming 'pipe' is your trained pipeline object
pickle.dump(pipe, open('pipe.pkl', 'wb'))

In [61]:
eligible_cities

['Colombo',
 'Mirpur',
 'Johannesburg',
 'Dubai',
 'Auckland',
 'Cape Town',
 'London',
 'Pallekele',
 'Barbados',
 'Sydney',
 'Melbourne',
 'Durban',
 'St Lucia',
 'Wellington',
 'Lauderhill',
 'Hamilton',
 'Centurion',
 'Manchester',
 'Abu Dhabi',
 'Mumbai',
 'Nottingham',
 'Southampton',
 'Mount Maunganui',
 'Chittagong',
 'Kolkata',
 'Lahore',
 'Delhi',
 'Nagpur',
 'Chandigarh',
 'Adelaide',
 'Bangalore',
 'St Kitts',
 'Cardiff',
 'Christchurch',
 'Trinidad']